In [1]:
!pip install -q ultralytics

In [2]:
import os
import glob
import pandas as pd
import torch
from ultralytics import YOLO

In [3]:
# mps or cuda
device = "cuda"

In [4]:
%%time

# 1. 테스트할 이미지 경로 (본인의 실제 테스트 이미지 경로로 꼭 바꿔주세요!)
source_path = 'img_294.png'


# 2. 모델 파일 찾기
model_files = glob.glob("models/*.pt")
model_files.sort()

benchmark_data = []

print(f"테스트 모델 목록: {model_files}")
print(f"옵션: device='{device}', half=True, 탐지 개수 카운트 포함\n")

for file in model_files:
    model_name = os.path.splitext(file)[0]
    print(f"[{model_name}] 벤치마크 진행 중...", end=" ")

    try:
        model = YOLO(file)

        # [Warm-up] 예열 10회
        for _ in range(10):
            model.predict(source=source_path, verbose=False, device=device, half=True)

        # [Benchmark] 본 테스트 100회
        inference_times = []
        preprocess_times = []
        postprocess_times = []
        detected_counts = []  # 탐지된 개수를 저장할 리스트

        for _ in range(100):
            results = model.predict(source=source_path, save=False, verbose=False, device='mps', half=True)

            # 1. 속도 측정
            speed = results[0].speed
            preprocess_times.append(speed['preprocess'])
            inference_times.append(speed['inference'])
            postprocess_times.append(speed['postprocess'])

            # 2. 탐지된 WBC 개수 카운트 (Boxes 개수)
            # 만약 특정 클래스(예: class 0)만 세야 한다면 results[0].boxes.cls를 필터링해야 합니다.
            # 여기서는 모든 탐지 객체를 WBC로 가정하고 전체 개수를 셉니다.
            count = len(results[0].boxes)
            detected_counts.append(count)

        # 평균 계산
        avg_infer = sum(inference_times) / len(inference_times)
        avg_pre = sum(preprocess_times) / len(preprocess_times)
        avg_post = sum(postprocess_times) / len(postprocess_times)
        avg_count = sum(detected_counts) / len(detected_counts) # 평균 탐지 개수

        total_time_ms = avg_pre + avg_infer + avg_post
        fps = 1000 / total_time_ms if total_time_ms > 0 else 0

        # 결과 데이터에 추가
        benchmark_data.append({
            "Model": model_name,
            "Mode": "FP16",
            "Avg Detected": round(avg_count, 1), # 평균 탐지 개수 (소수점 1자리)
            "Inference (ms)": round(avg_infer, 2),
            "Total (ms)": round(total_time_ms, 2),
            "FPS": round(fps, 2)
        })

        print(f"완료! (평균 {avg_count:.1f}개 탐지)")

    except Exception as e:
        print(f"실패 ({e})")

# 3. 결과 테이블 출력
if benchmark_data:
    df = pd.DataFrame(benchmark_data)
    df = df.sort_values(by="FPS", ascending=False)

    print("\n" + "="*70)
    print("   " + torch.cuda.get_device_name(0))
    print("="*70)
    print("   YOLO 모델 종합 성능 비교 (속도 vs 탐지 개수)")
    print("="*70)
    display(df)
else:
    print("모델 파일을 찾지 못했습니다.")

테스트 모델 목록: ['models/rt_detr.pt', 'models/yolo10.pt', 'models/yolo10s.pt', 'models/yolon.pt', 'models/yolos.pt']
옵션: device='cuda', half=True, 탐지 개수 카운트 포함

[models/rt_detr] 벤치마크 진행 중... 

/home/jychun/miniconda3/envs/wbc/lib/python3.11/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


완료! (평균 17.0개 탐지)
[models/yolo10] 벤치마크 진행 중... 완료! (평균 10.0개 탐지)
[models/yolo10s] 벤치마크 진행 중... 완료! (평균 12.0개 탐지)
[models/yolon] 벤치마크 진행 중... 완료! (평균 6.0개 탐지)
[models/yolos] 벤치마크 진행 중... 완료! (평균 12.0개 탐지)

   NVIDIA GB10
   YOLO 모델 종합 성능 비교 (속도 vs 탐지 개수)


,Model,Mode,Avg Detected,Inference (ms),Total (ms),FPS
1,models/yolo10,FP16,10.0,3.67,6.35,157.38
2,models/yolo10s,FP16,12.0,3.71,6.79,147.37
3,models/yolon,FP16,6.0,2.62,9.98,100.17
4,models/yolos,FP16,12.0,2.88,12.22,81.84
0,models/rt_detr,FP16,17.0,15.03,19.29,51.83


CPU times: user 1min 11s, sys: 14.1 s, total: 1min 25s
Wall time: 1min 26s
